Import libraries

In [34]:
import numpy as np
import scipy as sp
import scipy.sparse as sparse
import cvxpy as cvx
import osqp
from time import time

import mosek

Define size of the problems

In [35]:
# Dimensions of the problem
n = 200
k = 50

In [36]:
# Generate problem data
sp.random.seed(1)
F     = sparse.random(n, k, density=0.7, format='csc')
D     = sparse.diags(np.random.rand(n) * np.sqrt(k), format='csc')
Sigma = F*F.T + D
mu    = np.random.randn(n)
gamma = 1

CVXPY

In [37]:
# Define problem
x = cvx.Variable(n)
objective = - mu.T*x + gamma*cvx.quad_form(x,Sigma)
constraints = [sum(x) == 1, x >= 0]

SCS

In [38]:
# Solve SCS
t = time()
cvx.Problem(cvx.Minimize(objective), constraints).solve(solver = 'SCS')
Tscs = time() - t

ECOS

In [39]:
# Solve ECOS
t = time()
cvx.Problem(cvx.Minimize(objective), constraints).solve(solver = 'ECOS')
Tecos = time() - t

OSQP solver

In [40]:
# Solve OSQPsolver
t = time()
cvx.Problem(cvx.Minimize(objective), constraints).solve(solver = cvx.OSQP)
Tosqps = time() - t

MOSEK

In [41]:
# # Solve MOSEK
# t = time()
# cvx.Problem(cvx.Minimize(objective), constraints).solve(solver = cvx.MOSEK)
# Tmosek = time() - t

Elemental

OSQP

In [293]:
# OSQP data
P = sparse.block_diag((D, sparse.eye(k)), format='csc')
q = np.hstack([-mu / (2*gamma), np.zeros(k)])
A = sparse.vstack([
        sparse.hstack([F.T, -sparse.eye(k)]),
        sparse.hstack([sparse.csc_matrix(np.ones((1, n))), sparse.csc_matrix((1, k))]),
        sparse.hstack((sparse.eye(n), sparse.csc_matrix((n, k))))
    ]).tocsc()
l = np.hstack([np.zeros(k), 1., np.zeros(n)])
u = np.hstack([np.zeros(k), 1., np.ones(n)])

In [294]:
# Create an OSQP object
prob = osqp.OSQP()

# Setup workspace
prob.setup(P, q, A, l, u)

# Solve problem
t = time()
res = prob.solve()
Tosqp = time() - t

-----------------------------------------------------------------
           OSQP v0.5.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2018
-----------------------------------------------------------------
problem:  variables n = 250, constraints m = 251
          nnz(P) + nnz(A) = 7700
settings: linear system solver = qdldl,
          eps_abs = 1.0e-03, eps_rel = 1.0e-03,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 4000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: off

iter   objective    pri res    dua res    rho        time
   1  -6.9151e+00   1.17e+00   1.14e+02   1.00e-01   3.42e-03s
 150   1.4379e+00   6.02e-04   8.40e-04   1.95e+00   4.75e-03s

status:               solved
number of iterations: 150
optim

In [295]:
print Tosqp
print Tosqps
print Tscs
print Tecos

0.00579309463501
0.0337760448456
0.104723930359
0.317884922028
0.313035011292


In [296]:
print Tosqps/Tosqp
print Tscs/Tosqp
print Tecos/Tosqp

5.83039756359
18.0773726233
54.8730759733
54.0358877274
